In [1]:
from pulp import LpMinimize, LpProblem, LpStatus, lpSum, LpVariable
import pulp

In [2]:
model = LpProblem(name="CostMinimization", sense=LpMinimize)
model_opt = LpProblem(name="CostMinimizationOpt", sense=LpMinimize)

# Variables

In [3]:
x = [LpVariable(name=f"x{i}", lowBound=0) for i in range(0, 12)]

In [4]:
y = [LpVariable(name=f"y{i}", lowBound=0, upBound=1, cat='Binary') for i in range(12)]

In [5]:
# standards
manufacturing_time_per_unit = 0.333
cost_of_hour = 12
storage_cost_per_unit = 3

# these are monthly
cost_of_raw_materials_per_unit = [11, 10, 13, 9, 8, 7,
                                  10, 12, 12, 10, 8, 9]
demand = [150, 200, 100, 300, 200,
          400, 300, 250, 150, 200, 300, 350]
avalaible_hours = [250, 250, 200, 150, 200, 200,
                   150, 200, 250, 150, 150, 200]

# Constraints

In [6]:
cost_sum = 0
cost_sum_opt = 0
stored = [100]

for i in range(1, 13):
    hours_constraint = manufacturing_time_per_unit*x[i-1] <= avalaible_hours[i-1]
    model += hours_constraint
    model_opt += hours_constraint
    demand_constraint = x[i-1] + stored[i-1] >= demand[i-1]
    model += demand_constraint
    model_opt += demand_constraint
    stored.append(x[i-1] + stored[i-1] - demand[i-1])

# First Model without Binary Variable

In [7]:
cost1 = 0
for i in range(0, 12):
    cost1 += cost_of_hour*manufacturing_time_per_unit*x[i]
            +stored[i]*storage_cost_per_unit
            +cost_of_raw_materials_per_unit[i]*x[i]

In [8]:
model += cost1

In [9]:
model.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/m8/41547t_d4vbb207z7w8gbgyr0000gn/T/7d4708689afa4ac8a06bad90bcb70c4c-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/m8/41547t_d4vbb207z7w8gbgyr0000gn/T/7d4708689afa4ac8a06bad90bcb70c4c-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 29 COLUMNS
At line 132 RHS
At line 157 BOUNDS
At line 158 ENDATA
Problem MODEL has 24 rows, 12 columns and 90 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 11 (-13) rows, 12 (0) columns and 77 (-13) elements
0  Obj 2399.8 Primal inf 15500 (11)
10  Obj 77388.8
Optimal - objective value 77388.8
After Postsolve, objective 77388.8, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 77388.8 - 10 iterations time 0.002, Presolve 

1

In [10]:
model.objective.value()

37788.80000000001

In [11]:
for i in range(0, 12):
    print(x[i].value())

50.0
300.0
0.0
300.0
200.0
400.0
300.0
250.0
150.0
200.0
300.0
350.0


In [12]:
model

CostMinimization:
MINIMIZE
47.996*x0 + 43.996*x1 + 14.996*x10 + 12.996*x11 + 43.996*x2 + 36.996*x3 + 32.996*x4 + 28.996000000000002*x5 + 28.996000000000002*x6 + 27.996000000000002*x7 + 24.996000000000002*x8 + 19.996000000000002*x9 + -39600.0
SUBJECT TO
_C1: 0.333 x0 <= 250

_C2: x0 >= 50

_C3: 0.333 x1 <= 250

_C4: x0 + x1 >= 250

_C5: 0.333 x2 <= 200

_C6: x0 + x1 + x2 >= 350

_C7: 0.333 x3 <= 150

_C8: x0 + x1 + x2 + x3 >= 650

_C9: 0.333 x4 <= 200

_C10: x0 + x1 + x2 + x3 + x4 >= 850

_C11: 0.333 x5 <= 200

_C12: x0 + x1 + x2 + x3 + x4 + x5 >= 1250

_C13: 0.333 x6 <= 150

_C14: x0 + x1 + x2 + x3 + x4 + x5 + x6 >= 1550

_C15: 0.333 x7 <= 200

_C16: x0 + x1 + x2 + x3 + x4 + x5 + x6 + x7 >= 1800

_C17: 0.333 x8 <= 250

_C18: x0 + x1 + x2 + x3 + x4 + x5 + x6 + x7 + x8 >= 1950

_C19: 0.333 x9 <= 150

_C20: x0 + x1 + x2 + x3 + x4 + x5 + x6 + x7 + x8 + x9 >= 2150

_C21: 0.333 x10 <= 150

_C22: x0 + x1 + x10 + x2 + x3 + x4 + x5 + x6 + x7 + x8 + x9 >= 2450

_C23: 0.333 x11 <= 200

_C24: x0 +

# Second Model with Binary Variable

In [13]:
for i in range(0, 12):
    model_opt += x[i] <= 1000*y[i]
    model_opt += x[i] >= y[i]

In [14]:
cost2 = 0
for i in range(0, 12):
    cost2 += 1000*y[i]
            +cost_of_hour*manufacturing_time_per_unit*x[i]
            +stored[i]*storage_cost_per_unit + cost_of_raw_materials_per_unit[i]*x[i]

In [15]:
model_opt += cost2

In [16]:
model_opt.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/m8/41547t_d4vbb207z7w8gbgyr0000gn/T/860555e3be5841b49463be88b5905905-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/m8/41547t_d4vbb207z7w8gbgyr0000gn/T/860555e3be5841b49463be88b5905905-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 53 COLUMNS
At line 240 RHS
At line 289 BOUNDS
At line 302 ENDATA
Problem MODEL has 48 rows, 24 columns and 138 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 80188.8 - 0.00 seconds
Cgl0004I processed model has 33 rows, 23 columns (11 integer (11 of which binary)) and 121 elements
Cbc0038I Initial state - 9 integers unsatisfied sum - 3.09857
Cbc0038I Pass   1: suminf.    0.46380 (1) obj. 93327.7 iterations 11
Cbc0

1

In [17]:
model_opt.objective.value()

47636.39760000001

In [18]:
model_opt

CostMinimizationOpt:
MINIMIZE
47.996*x0 + 43.996*x1 + 14.996*x10 + 12.996*x11 + 43.996*x2 + 36.996*x3 + 32.996*x4 + 28.996000000000002*x5 + 28.996000000000002*x6 + 27.996000000000002*x7 + 24.996000000000002*x8 + 19.996000000000002*x9 + 1000*y0 + 1000*y1 + 1000*y10 + 1000*y11 + 1000*y2 + 1000*y3 + 1000*y4 + 1000*y5 + 1000*y6 + 1000*y7 + 1000*y8 + 1000*y9 + -39600.0
SUBJECT TO
_C1: 0.333 x0 <= 250

_C2: x0 >= 50

_C3: 0.333 x1 <= 250

_C4: x0 + x1 >= 250

_C5: 0.333 x2 <= 200

_C6: x0 + x1 + x2 >= 350

_C7: 0.333 x3 <= 150

_C8: x0 + x1 + x2 + x3 >= 650

_C9: 0.333 x4 <= 200

_C10: x0 + x1 + x2 + x3 + x4 >= 850

_C11: 0.333 x5 <= 200

_C12: x0 + x1 + x2 + x3 + x4 + x5 >= 1250

_C13: 0.333 x6 <= 150

_C14: x0 + x1 + x2 + x3 + x4 + x5 + x6 >= 1550

_C15: 0.333 x7 <= 200

_C16: x0 + x1 + x2 + x3 + x4 + x5 + x6 + x7 >= 1800

_C17: 0.333 x8 <= 250

_C18: x0 + x1 + x2 + x3 + x4 + x5 + x6 + x7 + x8 >= 1950

_C19: 0.333 x9 <= 150

_C20: x0 + x1 + x2 + x3 + x4 + x5 + x6 + x7 + x8 + x9 >= 2150

_C

In [19]:
for i in range(0, 12):
    print(x[i].value())

50.0
300.0
0.0
300.0
299.3994
600.6006
0.0
400.0
0.0
200.0
300.0
350.0


In [20]:
for i in range(0, 12):
    print(y[i].value())

1.0
1.0
0.0
1.0
1.0
1.0
0.0
1.0
0.0
1.0
1.0
1.0
